## Demonstração do Neo4j usando o py2neo e um grafo baseado nos personagens do filme Star Wars.

Fonte: Adaptado de https://solutiondesign.com/blog/-/blogs/navigating-the-star-wars-universe-with-neo-1 (Andy McGrath)


#### Acesso a base de dados do Neo4j

In [1]:
from py2neo import Graph, authenticate, DBMS
authenticate("localhost:7474", "neo4j", "123")
graph = Graph("http://localhost:7474/db/data/")
my_dbms = DBMS("http://localhost:7474")
my_dbms.kernel_version

(3, 0, 7)

#### Deleção de todos os registros do banco de dados.

In [2]:
query = """
MATCH (n) DETACH DELETE n
"""

results = graph.run(query)
results

#### Criação da base de dados considerada, baseada no filme Star Wars.

In [3]:
query = """
CREATE
(LUKE {name:"Luke Skywalker"}),
(HAN {name:"Han Solo"}),
(LEIA {name:"Princess Leia Organa"}),
(OBI_WAN {name:"Obi-Wan Kenobi"}),
(YODA {name : "Yoda"}),
(TARKIN {name:"Grand Moff Tarkin"}),
(VADER {name:"Dark Vader"}),
(C3PO {name:"C3PO", droid:true}),
(R2D2 {name:"R2D2", droid:true}),
(BOBA_FETT {name:"Boba Fett"}),
(JABBA {name:"Jabba the Hut"}),
(GREEDO {name:"Greedo"}),
(CHEWBACCA {name:"Chewbacca"}),

(TATOOINE {name:"Tatooine", distance:13184}),
(DAGOBAH {name:"Dagobah", distance:15407}),
(ALDERAAN {name:"Alderaan", distance:1502}),
(CORUSCANT {name:"Coruscant", distance:3066}),
(CORELLIA {name:"Corellia", distance:2453}),
(KASHYYYK {name:"Kashyyyk", distance:9811}),

(JEDI {name:"Jedi"}),
(SITH {name:"Sith"}),
(REBELLION {name:"Rebellion"}),
(EMPIRE {name:"Empire"}),
(HIGHEST_BIDDER {name:"Highest Bidder"}),
(DARK_SIDE {name:"Dark Side"}),
(LIGHT_SIDE {name:"Light Side"}),

(LUKE)-[:FRIENDS_WITH]->(HAN),
(LUKE)-[:FRIENDS_WITH]->(LEIA),
(HAN)-[:FRIENDS_WITH]->(CHEWBACCA),

(YODA)-[:TEACHES]->(OBI_WAN),
(YODA)-[:TEACHES]->(LUKE),
(OBI_WAN)-[:TEACHES]->(LUKE),

(OBI_WAN)-[:KNOWS]->(VADER),
(TARKIN)-[:KNOWS]->(VADER),
(VADER)-[:KNOWS]->(YODA),
(VADER)-[:KNOWS]->(LEIA),
(LEIA)-[:KNOWS]->(LUKE),
(LUKE)-[:KNOWS]->(YODA),
(TARKIN)-[:KNOWS]->(BOBA_FETT),
(BOBA_FETT)-[:KNOWS]->(JABBA),
(JABBA)-[:KNOWS]->(GREEDO),
(LUKE)-[:KNOWS]->(R2D2),
(R2D2)-[:KNOWS]->(C3PO),
(VADER)-[:KNOWS]->(C3PO),

(LUKE)-[:LIVED_ON]->(TATOOINE),
(HAN)-[:LIVED_ON]->(CORELLIA),
(LEIA)-[:LIVED_ON]->(ALDERAAN),
(YODA)-[:LIVED_ON]->(DAGOBAH),
(VADER)-[:LIVED_ON]->(TATOOINE),
(CHEWBACCA)-[:LIVED_ON]->(KASHYYYK),
(OBI_WAN)-[:LIVED_ON]->(TATOOINE),
(TARKIN)-[:LIVED_ON]->(CORUSCANT),

(LUKE)-[:DEVOTED_TO]->(JEDI),
(LUKE)-[:DEVOTED_TO]->(REBELLION),
(LUKE)-[:DEVOTED_TO]->(LIGHT_SIDE),
(VADER)-[:DEVOTED_TO]->(SITH),
(VADER)-[:DEVOTED_TO]->(EMPIRE),
(VADER)-[:DEVOTED_TO]->(DARK_SIDE),
(LEIA)-[:DEVOTED_TO]->(REBELLION),
(HAN)-[:DEVOTED_TO]->(REBELLION),
(TARKIN)-[:DEVOTED_TO]->(EMPIRE),
(BOBA_FETT)-[:DEVOTED_TO]->(HIGHEST_BIDDER),
(OBI_WAN)-[:DEVOTED_TO]->(JEDI),
(OBI_WAN)-[:DEVOTED_TO]->(REBELLION),
(OBI_WAN)-[:DEVOTED_TO]->(LIGHT_SIDE),
(YODA)-[:DEVOTED_TO]->(JEDI),
(YODA)-[:DEVOTED_TO]->(REBELLION),
(YODA)-[:DEVOTED_TO]->(LIGHT_SIDE)
"""

graph.run(query)

#### Consulta de todas as relações associadas ao "Obi-Wan Kenobi".

In [5]:
import pandas as pd

query = """
MATCH (c{name:"Obi-Wan Kenobi"})-[r]-(m) 
RETURN c.name, type(r), m.name;
"""

results = graph.run(query).data()

df = pd.DataFrame(results)
df

,c.name,m.name,type(r)
0,Obi-Wan Kenobi,Luke Skywalker,TEACHES
1,Obi-Wan Kenobi,Dark Vader,KNOWS
2,Obi-Wan Kenobi,Yoda,TEACHES
3,Obi-Wan Kenobi,Light Side,DEVOTED_TO
4,Obi-Wan Kenobi,Jedi,DEVOTED_TO
5,Obi-Wan Kenobi,Rebellion,DEVOTED_TO
6,Obi-Wan Kenobi,Tatooine,LIVED_ON


#### Consulta de todos os personagens que foram mestres ou aprendizes do "Obi-Wan Kenobi".
- Se ficou em dúvida em relação ao Yoda ter treinado o Obi-Wan, veja http://starwars.wikia.com/wiki/Obi-wan.

In [6]:
query = """
MATCH (c{name:"Obi-Wan Kenobi"})-[:TEACHES]-(d) 
RETURN DISTINCT  d.name;
"""

results = graph.run(query).data()

df = pd.DataFrame(results)
df

,d.name
0,Luke Skywalker
1,Yoda


 #### Consulta dos conhecidos dos conhecidos (Friend-Of-Friend) do "Obi-Wan Kenobi".

In [7]:
query = """
MATCH (c{name:"Obi-Wan Kenobi"})-[:TEACHES|FRIENDS_WITH]-(m)-[:KNOWS|TEACHES|FRIENDS_WITH]-(d) 
RETURN DISTINCT  d.name;
"""

results = graph.run(query).data()

df = pd.DataFrame(results)
df

,d.name
0,Yoda
1,Han Solo
2,Princess Leia Organa
3,R2D2
4,Luke Skywalker
5,Dark Vader


#### Retirando da consulta anterior os inimigos (Siths).

In [8]:
query = """
MATCH (c{name:"Obi-Wan Kenobi"})-[:TEACHES|FRIENDS_WITH]-(m)-[:KNOWS|TEACHES|FRIENDS_WITH]-(d) 
WHERE NOT ((d)-[]-({name:"Sith"}))
RETURN DISTINCT  d.name;
"""

results = graph.run(query).data()

df = pd.DataFrame(results)
df

,d.name
0,Yoda
1,Han Solo
2,Princess Leia Organa
3,R2D2
4,Luke Skywalker
